In [ ]:
pip install transformers

In [ ]:
pip install ktrain

In [ ]:
#general purpose packages
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

#data processing
import re, string
import nltk

from sklearn import preprocessing
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split


#Naive Bayes
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

seed=42

#transfromers
from transformers import BertTokenizerFast
from transformers import TFBertModel

#metrics
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# Check the versions of libraries

# Python version
import sys
print('Python: {}'.format(sys.version))
# scipy
import scipy
print('scipy: {}'.format(scipy.__version__))
# numpy
import numpy
print('numpy: {}'.format(numpy.__version__))
# matplotlib
import matplotlib
print('matplotlib: {}'.format(matplotlib.__version__))
# pandas
import pandas
print('pandas: {}'.format(pandas.__version__))
# scikit-learn
import sklearn
print('sklearn: {}'.format(sklearn.__version__))# Load libraries
import pandas as pd
from pandas import read_csv
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
...

Python: 3.10.6 (main, May 29 2023, 11:10:38) [GCC 11.3.0]
scipy: 1.10.1
numpy: 1.22.4
matplotlib: 3.7.1
pandas: 1.5.3
sklearn: 1.2.2


Ellipsis

In [ ]:
df = pd.read_csv("labeled_data.csv")

In [ ]:
df

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,1,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,25292,3,0,1,2,1,"you've gone and broke the wrong heart baby, an..."
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies


In [ ]:
import nltk

In [ ]:
TEXT_CLEANING_RE = "!@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import re

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
stop_words = stopwords.words("english")
stemmer = SnowballStemmer("english")

In [ ]:
df['labels']=df['class'].map({0:'Hate Speech',1:'Offensive',2:'Neither'})
print(df.head)

<bound method NDFrame.head of        Unnamed: 0  count  hate_speech  offensive_language  neither  class  \
0               0      3            0                   0        3      1   
1               1      3            0                   3        0      1   
2               2      3            0                   3        0      1   
3               3      3            0                   2        1      1   
4               4      6            0                   6        0      1   
...           ...    ...          ...                 ...      ...    ...   
24778       25291      3            0                   2        1      1   
24779       25292      3            0                   1        2      1   
24780       25294      3            0                   3        0      1   
24781       25295      6            0                   6        0      1   
24782       25296      3            0                   0        3      1   

                                             

In [ ]:
import string

In [ ]:
df=df[['tweet','labels']]

In [ ]:
def clean(text):
    text=str(text).lower()
    text=re.sub('\[.*?/].','',text)
    text=re.sub('https?://\S+|www.\.\S+','',text)
    text=re.sub('<.*?>+','',text)
    text=re.sub('[%s]'% re.escape(string.punctuation),'',text)
    text=re.sub('\w*\d\w*','',text)
    text=[word for word in text.split(' ')if word not in stop_words]
    text=" ".join(text)
    return text
df['tweet']=df['tweet'].apply(clean)
df

<ipython-input-77-5f01c163a39f>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tweet']=df['tweet'].apply(clean)


,tweet,labels
0,rt mayasolovely woman shouldnt complain clean...,Offensive
1,rt boy dats coldtyga dwn bad cuffin dat hoe ...,Offensive
2,rt urkindofbrand dawg rt ever fuck bitch sta...,Offensive
3,rt cganderson vivabased look like tranny,Offensive
4,rt shenikaroberts shit hear might true might ...,Offensive
...,...,...
24778,yous muthafin lie coreyemanuel right tl tras...,Offensive
24779,youve gone broke wrong heart baby drove rednec...,Offensive
24780,young buck wanna eat dat nigguh like aint fuck...,Offensive
24781,youu got wild bitches tellin lies,Offensive


In [ ]:
X=np.array(df['tweet'])
y=np.array(df['labels'])
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, stratify=y, random_state=1)
y.shape


(24783,)

In [ ]:
df.dropna(inplace=True)
X_test = df['tweet'].values
y_test = df['labels'].values

<ipython-input-79-cbd2c44e118d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


In [ ]:
from sklearn import preprocessing
ohe = preprocessing.OneHotEncoder()
Y_train = ohe.fit_transform(np.array(y_train).reshape(-1, 1))
y_valid = ohe.fit_transform(np.array(y_valid).reshape(-1, 1))
Y_test = ohe.fit_transform(np.array(y_test).reshape(-1, 1))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
#Naive Bayes
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import KFold
KFold

clf = CountVectorizer()
X_train_cv =  clf.fit_transform(X_train)
X_test_cv = clf.transform(X_test)
X_valid_cv = clf.transform(X_valid)


tf_transformer = TfidfTransformer(use_idf=True).fit(X_train_cv)
X_train_tf = tf_transformer.transform(X_train_cv)
X_valid_tf = tf_transformer.transform(X_valid_cv)


In [ ]:
y_test.shape

(24783,)

In [ ]:
y_train_le = y_train.copy()
y_valid_le = y_valid.copy()
y_test_le = y_test.copy()

In [ ]:
import time
# store starting time
begin = time.time()
models = []
models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))

models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))

results = []
names = []
for name, model in models:
	kfold = KFold(n_splits=10, random_state=1, shuffle=True)
	cv_results = cross_val_score(model, X_train_tf, y_train_le, cv=kfold, scoring='accuracy')
	results.append(cv_results)
	names.append(name)
	print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

end = time.time()

# total time taken
print(f"Total runtime : {end - begin}")


LR: 0.944181 (0.003451)
KNN: 0.940101 (0.003750)
CART: 0.924811 (0.009612)
Total runtime : 181.21315169334412


In [ ]:
uv = ['Hate Speech', 'Offensive', 'Neither']

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tag import pos_tag

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

def extract_pos_features(text):
    # Tokenize text into sentences
    sentences = sent_tokenize(text)

    pos_features = []
    stop_words = set(stopwords.words('english'))  # Set of common English stopwords

    for sentence in sentences:
        # Tokenize sentence into words
        words = word_tokenize(sentence)

        # Filter out stopwords and punctuation
        words = [word.lower() for word in words if word.isalpha() and word.lower() not in stop_words]

        # Perform POS tagging
        pos_tags = pos_tag(words)

        # Extract POS features
        pos_counts = {}
        for word, pos in pos_tags:
            if pos in pos_counts:
                pos_counts[pos] += 1
            else:
                pos_counts[pos] = 1

        pos_features.append(pos_counts)

    return pos_features

# Example usage
hate_speech_text = "I hate you! You are such a terrible person."
pos_features = extract_pos_features(hate_speech_text)
print(pos_features)


[{'NN': 1}, {'JJ': 1, 'NN': 1}]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tag import pos_tag

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

def extract_pos_features(text):
    # Tokenize text into sentences
    sentences = sent_tokenize(text)

    pos_features = []
    stop_words = set(stopwords.words('english'))  # Set of common English stopwords

    for sentence in sentences:
        # Tokenize sentence into words
        words = word_tokenize(sentence)

        # Filter out stopwords and punctuation
        words = [word.lower() for word in words if word.isalpha() and word.lower() not in stop_words]

        # Perform POS tagging
        pos_tags = pos_tag(words)

        # Extract POS features
        pos_counts = {}
        for word, pos in pos_tags:
            if pos in pos_counts:
                pos_counts[pos] += 1
            else:
                pos_counts[pos] = 1

        pos_features.append(pos_counts)

    return pos_features


# Load the dataset
dataset = pd.read_csv('labeled_data.csv')

# Extract POS features for each tweet in the dataset
pos_features_list = []
for tweet in dataset['tweet']:
    pos_features = extract_pos_features(tweet)
    pos_features_list.append(pos_features)

# Add the POS features as new columns to the dataset
pos_feature_names = set()
for pos_features in pos_features_list:
    for pos_counts in pos_features:
        pos_feature_names.update(pos_counts.keys())

for pos_feature_name in pos_feature_names:
    dataset[pos_feature_name] = 0  # Initialize the new columns with 0

for i, pos_features in enumerate(pos_features_list):
    for pos_counts in pos_features:
        dataset.loc[i, pos_counts.keys()] = pos_counts.values()

# Save the updated dataset with POS features
dataset.to_csv('dataset_with_pos_features.csv', index=False)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
dataset

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet,UH,RBR,RP,...,NN,SYM,VBP,JJS,TO,CC,WDT,EX,NNPS,VBD
0,0,3,0,0,3,1,!!! RT @mayasolovely: As a woman you shouldn't...,0,0,0,...,2,0,1,0,0,0,0,0,0,0
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,0,0,0,...,7,0,1,0,0,0,0,0,0,0
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,0,0,0,...,1,0,0,0,0,0,0,0,0,1
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,0,0,0,...,2,0,0,0,0,0,0,0,0,0
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,0,0,0,...,3,0,0,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,0,0,0,...,1,0,1,0,0,0,0,0,0,0
24779,25292,3,0,1,2,1,"you've gone and broke the wrong heart baby, an...",0,0,0,...,4,0,0,0,0,0,0,0,0,2
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...,0,0,0,...,3,0,0,0,0,0,0,0,0,0
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies,0,0,0,...,2,0,0,0,0,0,0,0,0,1


In [ ]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

def word_embedding_feature_extraction(dataset):
    # Create a Word2Vec model
    sentences = [word_tokenize(tweet) for tweet in dataset['tweet']] #set of tweets that have been tokenized
    model = Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=1)

    # Create a feature vector for each tweet
    feature_vectors = []
    for tweet in dataset['tweet']:
        tweet_vector = np.zeros(100)
        for word in word_tokenize(tweet):
            if word in model.wv.key_to_index:
                tweet_vector += model.wv[word]
        feature_vectors.append(tweet_vector)

    return feature_vectors

# Load the dataset
dataset = pd.read_csv('labeled_data.csv')

# Extract the word embedding features
feature_vectors = word_embedding_feature_extraction(dataset)

# Save the feature vectors to a file
np.save('word_embedding_features.npy', feature_vectors)


In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
# Load the word embeddings
word_embeddings = np.load("word_embedding_features.npy")

# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(word_embeddings, dataset["class"], test_size=0.25)

# Train the logistic regression model
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

model2 = LogisticRegression()
model2.fit(X_train,y_train)

model3 = KNeighborsClassifier()
model3.fit(X_train,y_train)

model4 = GaussianNB()
# Evaluate the NB model
model4.fit(X_train , y_train)

model5 = RandomForestClassifier()
#Evaluate the RandomForest Model
model5.fit(X_train , y_train)

model6 = SVC()
#Evaluate the SVM model
model6.fit(X_train , y_train)

model7 = AdaBoostClassifier(learning_rate = 1)
#Evaluate the AdaBoost Classifier
model7.fit(X_train , y_train)

y_predict = model.predict(X_test)
y_predict2 = model2.predict(X_test)
y_predict3 = model3.predict(X_test)
y_predict4 = model4.predict(X_test)
y_predict5 = model5.predict(X_test)
y_predict6 = model6.predict(X_test)
y_predict7 = model7.predict(X_test)

score = model.score(X_test, y_test)
score2 = model2.score(X_test, y_test)
score3 = model3.score(X_test, y_test)
score4 = model4.score(X_test, y_test)
score5 = model5.score(X_test, y_test)
score6 = model6.score(X_test, y_test)
score7 = model7.score(X_test, y_test)

classification_rep = classification_report(y_test,y_predict)
classification_rep2 = classification_report(y_test,y_predict2)
classification_rep3 = classification_report(y_test,y_predict3)
classification_rep4 = classification_report(y_test,y_predict4)
classification_rep5 = classification_report(y_test,y_predict5)
classification_rep6 = classification_report(y_test,y_predict6)
classification_rep7 = classification_report(y_test,y_predict7)

print('LR:', score2)
print('KNN:', score3)
print('CART:', score)
print('NB:',score4)
print('RF:',score5)
print('SVM:',score6)
print('AdaBoost:',score7)

print(classification_rep)
print(classification_rep2)
print(classification_rep3)
print(classification_rep4)
print(classification_rep5)
print(classification_rep6)
print(classification_rep7)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LR: 0.9401226597805036
KNN: 0.9401226597805036
CART: 0.8841187863137508
NB: 0.24919302775984506
RF: 0.9406068431245965
SVM: 0.9414138153647514
AdaBoost: 0.9410910264686895
              precision    recall  f1-score   support

           0       0.11      0.14      0.12       363
           1       0.95      0.93      0.94      5833

    accuracy                           0.88      6196
   macro avg       0.53      0.53      0.53      6196
weighted avg       0.90      0.88      0.89      6196

              precision    recall  f1-score   support

           0       0.17      0.01      0.01       363
           1       0.94      1.00      0.97      5833

    accuracy                           0.94      6196
   macro avg       0.55      0.50      0.49      6196
weighted avg       0.90      0.94      0.91      6196

              precision    recall  f1-score   support

           0       0.39      0.04      0.07       363
           1       0.94      1.00      0.97      5833

    accura

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import numpy as np
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
def sentiment_analysis(tweet):
    sid = SentimentIntensityAnalyzer()
    score = sid.polarity_scores(tweet)
    sentiment_scores = np.array([score["neg"], score["neu"], score["pos"], score["compound"]])
    return sentiment_scores

# Load the dataset
dataset = pd.read_csv("labeled_data.csv")

# Extract the sentiment scores
sentiment_scores = []
for tweet in dataset["tweet"]:
    score = sentiment_analysis(tweet)
    sentiment_scores.append(score)

# Save the sentiment scores
np.save("sentiment_scores.npy", sentiment_scores)

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
# Load the sentiment scores
sentiment_scores = np.load("sentiment_scores.npy", allow_pickle="False")

# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(sentiment_scores, dataset["class"], test_size=0.25)
# Create a logistic regression model
model = LogisticRegression()
# Train the logistic regression model
model.fit(X_train, y_train)

model2 = DecisionTreeClassifier()
# Evaluate the Decision Tree model
model2.fit(X_train, y_train)

model3 = KNeighborsClassifier()
# Evaluate the KNN model
model3.fit(X_train, y_train)

model4 = GaussianNB()
# Evaluate the NB model
model4.fit(X_train , y_train)

model5 = RandomForestClassifier()
#Evaluate the RandomForest Model
model5.fit(X_train , y_train)

model6 = SVC()
#Evaluate the SVM model
model6.fit(X_train , y_train)

model7 = AdaBoostClassifier(learning_rate = 1)
#Evaluate the AdaBoost Classifier
model7.fit(X_train , y_train)

y_predict = model.predict(X_test)
y_predict2 = model2.predict(X_test)
y_predict3 = model3.predict(X_test)
y_predict4 = model4.predict(X_test)
y_predict5 = model5.predict(X_test)
y_predict6 = model6.predict(X_test)
y_predict7 = model7.predict(X_test)
score = model.score(X_test, y_test)
score2 = model2.score(X_test, y_test)
score3 = model3.score(X_test, y_test)
score4 = model4.score(X_test, y_test)
score5 = model5.score(X_test, y_test)
score6 = model6.score(X_test, y_test)
score7 = model7.score(X_test, y_test)
classification_rep = classification_report(y_test,y_predict)
classification_rep2 = classification_report(y_test,y_predict2)
classification_rep3 = classification_report(y_test,y_predict3)
classification_rep4 = classification_report(y_test,y_predict4)
classification_rep5 = classification_report(y_test,y_predict5)
classification_rep6 = classification_report(y_test,y_predict6)
classification_rep7 = classification_report(y_test,y_predict7)
print('LR:', score)
print('CART:',score2)
print('KNN:',score3)
print('NB:',score4)
print('RF:',score5)
print('SVM:',score6)
print('AdaBoost:',score7)
#evaluate_model(model,X_test,y_test,dataset["class"])
print(classification_rep)
print(classification_rep2)
print(classification_rep3)
print(classification_rep4)
print(classification_rep5)
print(classification_rep6)
print(classification_rep7)
#accuracy of LR and SVM are same as the data is linearly separable i.e. in a 2d space we can separate data points into two classes

LR: 0.9388315041962556
CART: 0.8996126533247256
KNN: 0.9373789541639768
NB: 0.9375403486120077
RF: 0.9289864428663654
SVM: 0.9388315041962556
AdaBoost: 0.9386701097482246
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       379
           1       0.94      1.00      0.97      5817

    accuracy                           0.94      6196
   macro avg       0.47      0.50      0.48      6196
weighted avg       0.88      0.94      0.91      6196

              precision    recall  f1-score   support

           0       0.10      0.08      0.09       379
           1       0.94      0.95      0.95      5817

    accuracy                           0.90      6196
   macro avg       0.52      0.52      0.52      6196
weighted avg       0.89      0.90      0.89      6196

              precision    recall  f1-score   support

           0       0.34      0.03      0.05       379
           1       0.94      1.00      0.97      5817

    accurac

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif